In [51]:
from database import database

database= database("data.db")
database.connect()
resultats=database.read_json("all_matches_of_Test6.docx.json")
resultats






,title,question,sentence,score,pts
0,Analyse et synthèse,La fonction requiert la faculté d'analyser des...,recherches et analyse dans un environnement co...,0.583461,42.757
1,Analyse et synthèse,La fonction requiert la faculté d'analyser des...,Développer et stimuler les compétences individ...,0.528830,29.601
2,Analyse et synthèse,La fonction requiert la faculté d'analyser des...,Élabore et développe des outils spécifiques à ...,0.496883,29.601
3,Analyse et synthèse,La fonction requiert la faculté d'analyser des...,en démontrant une forte aptitude dans la relat...,0.478152,29.601
4,Analyse et synthèse,La fonction requiert la faculté d'analyser des...,la formatrice veille à acquérir de nouvelles c...,0.426242,29.601
...,...,...,...,...,...
187,Rôle dans la gestion des ressources humaines,Elle a un rôle d'animation incluant le transfe...,Élabore des activités pour la formation pratique,0.559728,11.232
188,Rôle dans la gestion des ressources humaines,Elle a un rôle d'animation incluant le transfe...,En tant que personne ressource dans des thémat...,0.532869,11.232
189,Rôle dans la gestion des ressources humaines,Elle a un rôle de conduite de collaborateurs/t...,Agir dans le respect de la charte du personnel...,0.480368,24.336
190,Rôle dans la gestion des ressources humaines,"Elle a un rôle de direction, d'organisation et...",(selon critères d'évaluation des fonctions des...,0.436894,37.440


In [52]:
import spacy
nlp = spacy.load("fr_core_news_md")  # pas besoin d'exclude

def lemmatize(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc])

def to_lower(text):
    return text.lower()


In [53]:
resultats["sentence_lemmatized"] = resultats["sentence"].apply(to_lower)
resultats["sentence_lemmatized"] = resultats["sentence_lemmatized"].apply(lemmatize)
resultats['question_lemmatized'] = resultats['question'].apply(to_lower)
resultats['question_lemmatized'] = resultats['question_lemmatized'].apply(lemmatize)
resultats

,title,question,sentence,score,pts,sentence_lemmatized,question_lemmatized
0,Analyse et synthèse,La fonction requiert la faculté d'analyser des...,recherches et analyse dans un environnement co...,0.583461,42.757,recherche et analyse dans un environnement com...,le fonction requérir le faculté de analyser un...
1,Analyse et synthèse,La fonction requiert la faculté d'analyser des...,Développer et stimuler les compétences individ...,0.528830,29.601,développer et stimuler le compétence individue...,le fonction requérir le faculté de analyser un...
2,Analyse et synthèse,La fonction requiert la faculté d'analyser des...,Élabore et développe des outils spécifiques à ...,0.496883,29.601,élaborer et développer un outil spécifique à l...,le fonction requérir le faculté de analyser un...
3,Analyse et synthèse,La fonction requiert la faculté d'analyser des...,en démontrant une forte aptitude dans la relat...,0.478152,29.601,en démontrer un fort aptitude dans le relation...,le fonction requérir le faculté de analyser un...
4,Analyse et synthèse,La fonction requiert la faculté d'analyser des...,la formatrice veille à acquérir de nouvelles c...,0.426242,29.601,le formateur veille à acquérir de nouveau conn...,le fonction requérir le faculté de analyser un...
...,...,...,...,...,...,...,...
187,Rôle dans la gestion des ressources humaines,Elle a un rôle d'animation incluant le transfe...,Élabore des activités pour la formation pratique,0.559728,11.232,élaborer un activité pour le formation pratique,lui avoir un rôle de animation inclure le tran...
188,Rôle dans la gestion des ressources humaines,Elle a un rôle d'animation incluant le transfe...,En tant que personne ressource dans des thémat...,0.532869,11.232,en tant que personne ressourc dans un thématiq...,lui avoir un rôle de animation inclure le tran...
189,Rôle dans la gestion des ressources humaines,Elle a un rôle de conduite de collaborateurs/t...,Agir dans le respect de la charte du personnel...,0.480368,24.336,agir dans le respect de le charte de personnel...,lui avoir un rôle de conduite de collaborateur...
190,Rôle dans la gestion des ressources humaines,"Elle a un rôle de direction, d'organisation et...",(selon critères d'évaluation des fonctions des...,0.436894,37.440,( selon critère de évaluation de fonction de é...,"lui avoir un rôle de direction , de organisati..."


In [54]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [55]:
KEYWORDS = {
    "Formation de base":          ["AFP", "CFC", "Bachelor", "Master", "doctorat", "université", "HES", "ES"],
    "Formation complémentaire":   ["complémentaire", "brevet", "CAS", "DAS", "MAS", "formations continues"],
    # "Durée":                      ["0", "1", "2", "3", "4", "5", "6", "7", "8", "ans", "année"],
    # "Nature":                     ["expérience", "préalable", "même type"],
    "Autonomie de décision":      ["consignes", "directives", "autonome", "approuve"],
    "Responsabilités budgétaires":["budget", "financier", "facturation", "paiements", "suivi"],
    # "Responsabilités de planification à court terme":
    #                                 ["court terme", "plan", "optimisation", "procédure"],
    # "Responsabilités de planification à long terme":
    #                                 ["long terme", "prospective", "anticiper", "recherches", "études"],
    # "Impact externe des prestations":
    #                                 ["image", "représentatif", "conséquences", "tiers"],
    # "Impact interne des prestations":
    #                                 ["coûts", "bon fonctionnement", "irréversibles"],
    # "Connaissances linguistiques":["français", "langue", "bilingue", "trilingue"],
    # "Nature des communications internes":["communications", "échanges", "négociations", "décisions"],
    # "Nature des communications externes":["communications", "informer", "explication", "négociations"],
    # "Complexité de l'environnement":["difficultés", "adaptabilité", "flexibilité"],
    # "Evolution de l'environnement":["évolution", "processus", "rapide"],
    # "Diversité des missions":     ["missions", "tâches", "diverses"],
    # "Diversité et quantité des postes à gérer":
    #                                 ["poste", "gère", "grand nombre", "activités"],
    # "Rôle dans la gestion des ressources humaines":
    #                                 ["animation", "conduite", "recrutement", "formation"],
    # "Innovation":                 ["adapter", "créer", "innovatrice", "concepts"]
}


In [56]:
import re

def contains_keywords(text, text2, keywords, title):
    text = text.lower()
    text2 = text2.lower()
    if title in keywords:
        for keyword in keywords[title]:
            pattern = r'\b' + re.escape(keyword.lower()) + r'\b'
            if re.search(pattern, text) and re.search(pattern, text2):
                print(f"Found keyword '{keyword}' in title '{title}'")
                return True
    return False

In [57]:

def tfidf_similarity(text1, text2):
    vect = TfidfVectorizer()
    tfidf = vect.fit_transform([text1, text2])

    return cosine_similarity(tfidf[0], tfidf[1])[0][0]

resultats

,title,question,sentence,score,pts,sentence_lemmatized,question_lemmatized
0,Analyse et synthèse,La fonction requiert la faculté d'analyser des...,recherches et analyse dans un environnement co...,0.583461,42.757,recherche et analyse dans un environnement com...,le fonction requérir le faculté de analyser un...
1,Analyse et synthèse,La fonction requiert la faculté d'analyser des...,Développer et stimuler les compétences individ...,0.528830,29.601,développer et stimuler le compétence individue...,le fonction requérir le faculté de analyser un...
2,Analyse et synthèse,La fonction requiert la faculté d'analyser des...,Élabore et développe des outils spécifiques à ...,0.496883,29.601,élaborer et développer un outil spécifique à l...,le fonction requérir le faculté de analyser un...
3,Analyse et synthèse,La fonction requiert la faculté d'analyser des...,en démontrant une forte aptitude dans la relat...,0.478152,29.601,en démontrer un fort aptitude dans le relation...,le fonction requérir le faculté de analyser un...
4,Analyse et synthèse,La fonction requiert la faculté d'analyser des...,la formatrice veille à acquérir de nouvelles c...,0.426242,29.601,le formateur veille à acquérir de nouveau conn...,le fonction requérir le faculté de analyser un...
...,...,...,...,...,...,...,...
187,Rôle dans la gestion des ressources humaines,Elle a un rôle d'animation incluant le transfe...,Élabore des activités pour la formation pratique,0.559728,11.232,élaborer un activité pour le formation pratique,lui avoir un rôle de animation inclure le tran...
188,Rôle dans la gestion des ressources humaines,Elle a un rôle d'animation incluant le transfe...,En tant que personne ressource dans des thémat...,0.532869,11.232,en tant que personne ressourc dans un thématiq...,lui avoir un rôle de animation inclure le tran...
189,Rôle dans la gestion des ressources humaines,Elle a un rôle de conduite de collaborateurs/t...,Agir dans le respect de la charte du personnel...,0.480368,24.336,agir dans le respect de le charte de personnel...,lui avoir un rôle de conduite de collaborateur...
190,Rôle dans la gestion des ressources humaines,"Elle a un rôle de direction, d'organisation et...",(selon critères d'évaluation des fonctions des...,0.436894,37.440,( selon critère de évaluation de fonction de é...,"lui avoir un rôle de direction , de organisati..."


In [58]:
similarities = []
for idx, row in resultats.iterrows():
    sim = tfidf_similarity(row['sentence_lemmatized'], row['question_lemmatized'])
    if contains_keywords(row['sentence'], row['question'], KEYWORDS, row['title']):
        sim += 1
    similarities.append(sim)
resultats['similarity'] = similarities
resultats=resultats.sort_values(by='similarity', ascending=False)
resultats_needed = resultats[["title", "question", "sentence", "score", "pts", "similarity"]]
resultats_needed.sort_values(by='similarity', ascending=False, inplace=True)
resultats_needed

Found keyword 'HES' in title 'Formation de base'


/var/folders/hw/687qrsn947v91l2yv331ngdc0000gn/T/ipykernel_73412/2835965532.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultats_needed.sort_values(by='similarity', ascending=False, inplace=True)


,title,question,sentence,score,pts,similarity
82,Formation de base,La fonction requiert un Bachelor (Université o...,Diplôme de travailleur-euse social-e de niveau...,0.572379,20.93000,1.148240
190,Rôle dans la gestion des ressources humaines,"Elle a un rôle de direction, d'organisation et...",(selon critères d'évaluation des fonctions des...,0.436894,37.44000,0.707992
119,Innovation,L'exercice de la fonction requiert la capacité...,Adapte le contenu de la formation selon les ré...,0.504594,34.74900,0.627742
11,Autonomie de décision,"La fonction approuve, pour une Fondation, les ...",Participe au choix et à l'élaboration d'un pro...,0.577818,117.00000,0.620794
18,Autonomie de décision,"La fonction approuve, pour une Fondation, les ...",de la charte de la Fondation,0.488221,117.00000,0.596301
...,...,...,...,...,...,...
88,Formation de base,La fonction requiert une formation post grade ...,Observation lors du stage d'admission,0.458801,36.40000,0.000000
85,Formation de base,La fonction requiert au minimum le niveau scol...,(formation de base,0.484864,6.37000,0.000000
109,Impact interne des prestations,L'activité de la fonction a un impact importan...,ses structures et son personnel,0.315266,14.32080,0.000000
163,Responsabilités de planification et de réalisa...,La fonction donne l'approbation finale sur les...,Gestion du projet,0.444172,77.21999,0.000000


In [59]:
resultats_fin = resultats_needed.sort_values('similarity', ascending=False).drop_duplicates(subset=['title'], keep='first')
resultats_fin

,title,question,sentence,score,pts,similarity
82,Formation de base,La fonction requiert un Bachelor (Université o...,Diplôme de travailleur-euse social-e de niveau...,0.572379,20.9300,1.148240
190,Rôle dans la gestion des ressources humaines,"Elle a un rôle de direction, d'organisation et...",(selon critères d'évaluation des fonctions des...,0.436894,37.4400,0.707992
119,Innovation,L'exercice de la fonction requiert la capacité...,Adapte le contenu de la formation selon les ré...,0.504594,34.7490,0.627742
11,Autonomie de décision,"La fonction approuve, pour une Fondation, les ...",Participe au choix et à l'élaboration d'un pro...,0.577818,117.0000,0.620794
112,Impact interne des prestations,L'activité de la fonction a un impact détermin...,Adapte le contenu de la formation selon les ré...,0.286584,28.6416,0.566126
98,Impact externe des prestations,La fonction a un rôle primordial dans le choix...,du système de gestion et de la Fondation dans ...,0.390202,55.5984,0.519093
178,Responsabilités de planification et de réalisa...,La fonction contribue de manière active à l'él...,Participe au choix et à l'élaboration d'un pro...,0.467523,11.9340,0.513511
52,Diversité et quantité des postes à gérer,La fonction n'a pas de postes directement subo...,Fonctions directement subordonnées,0.612161,0.0000,0.449436
169,Responsabilités de planification et de réalisa...,La fonction participe de manière prépondérante...,des infrastructures et du lieu de formation da...,0.405686,38.6100,0.439540
127,Nature,La fonction requiert d'avoir déjà eu le même t...,enseignant en école professionnel (IFFP ou équ...,0.545441,21.4500,0.431976


In [60]:
resultats_fin.to_json("results/after_comparaison_lexicale_of_Test6.docx.json", orient="records", force_ascii=False, indent=4)